<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4b3c5e61e60bfa52578466c21acdfa1753cfffdf67d1b0948018b2b374560343
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-03 10:40:06
-------------------
qualified stocks: 92
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.17 L
Current:  1.40 C
-------------------
Today PnL: 59.22 K (0.43%)
Current PnL: -17.45 L (-11.74%)
CY Booked + Current PnL: -8.41 L (-5.66%)
-------------------
Total profit:  3.55 L
Total loss:  -21.00 L
-------------------
Total Booked + Current PnL: 18.57 L (15.26%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.47%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.12 L (56.6%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-16.60,63.0,H-LC,8.19,115352.0,11040.0,5606.0,0.19,10.58,4.86,15.96,9.0,1.97,0.84,29.27,X5K,ATH,METALS
53,MEDANTA,1486.00,15.71,63.0,H-MC,15.61,165172.0,38972.0,7201.0,2.41,30.88,4.36,36.59,108.0,5.41,1.20,47.32,XY24,NTT,HEALTHCARE
26,DIXON,18940.15,-29.21,69.0,H-MC,6.81,141624.0,26688.0,9900.0,0.85,23.22,6.99,31.83,81.0,2.70,1.03,46.80,X40N,ATH,IT
82,TTKPRESTIG,770.00,106.62,69.0,M-SC,6.98,89034.0,-11743.0,11833.0,-0.34,-11.65,13.29,0.09,245.0,-0.99,0.65,16.97,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-14.97,57.0,H-LC,8.93,169928.0,13751.0,12337.0,0.55,8.80,7.26,16.71,22.0,1.11,1.23,27.78,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,731.35,5.57,46.0,H-SC,11.09,125786.0,489.0,58516.0,1.44,0.39,46.52,47.09,141.0,0.01,0.91,57.43,MH,ATH,POWER
39,ICICIPRULI,790.00,-20.54,40.0,H-MC,2.17,136651.0,863.0,41884.0,-0.98,0.64,30.65,31.48,107.0,0.02,0.99,12.83,X40,ATH,INSURANCE
32,HAVELLS,2069.16,0.20,64.0,H-MC,2.27,248461.0,730.0,74340.0,0.63,0.29,29.92,30.30,92.0,0.01,1.81,13.63,X40,ATH,ELECTRICAL


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,JIOFIN,387.00,-1.80,39.0,H-LC,14.25,217245.0,-161.0,53660.0,-0.45,-0.07,24.70,24.61,48.0,-0.00,1.58,54.70,XY24,BTT,FINANCE
33,HCLTECH,1943.91,0.22,40.0,H-LC,9.93,229032.0,-12892.0,76153.0,-0.47,-5.33,33.25,26.15,8.0,-0.17,1.66,8.23,X40,ATH,IT
61,RELIANCE,1533.00,-14.79,41.0,H-LC,7.65,213127.0,2761.0,26023.0,-0.02,1.31,12.21,13.68,37.0,0.11,1.55,18.05,XY25,NTT,REFINERIES
43,INFY,2275.00,-19.24,45.0,H-LC,11.30,314324.0,1436.0,170238.0,-1.53,0.46,54.16,54.87,3.0,0.01,2.28,7.14,X40,BTT,IT
79,TCS,4998.00,-26.76,48.0,H-LC,14.72,291344.0,-54612.0,178477.0,-0.39,-15.79,61.26,35.80,1.0,-0.31,2.12,3.21,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.20,-6.07,52.0,H-LC,2.35,233235.0,-12916.0,92128.0,-1.15,-5.25,39.50,32.18,16.0,-0.14,1.69,28.90,X200,ATH,IT
55,NESTLEIND,1377.00,-8.73,62.0,H-LC,3.10,280683.0,15257.0,41541.0,-0.12,5.75,14.80,21.40,11.0,0.37,2.04,13.98,XY25,NTT,FMCG
45,ITC,452.00,-37.92,50.0,H-LC,3.29,199952.0,-186.0,21075.0,0.53,-0.09,10.54,10.44,4.0,-0.01,1.45,5.47,X40,NTT,FMCG
20,CIPLA,1795.00,-18.75,59.0,H-LC,5.11,217118.0,12618.0,28790.0,0.99,6.17,13.26,20.25,10.0,0.44,1.58,15.44,X40N,BTT,PHARMA
88,VBL,671.64,-9.80,55.0,H-LC,7.44,324742.0,8900.0,103755.0,1.96,2.82,31.95,35.67,5.0,0.09,2.36,17.06,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,RELAXO,1176.00,-30.97,64.0,H-SC,6.30,64070.0,-51468.0,86456.0,2.87,-44.55,134.94,30.28,136.0,-0.60,0.47,25.96,X40N,NTT,FOOTWEAR
52,MASFIN,398.61,-16.44,57.0,H-SC,13.27,94725.0,-3255.0,24856.0,2.87,-3.32,26.24,22.05,152.0,-0.13,0.69,37.69,XR,ATH,FINANCE
53,MEDANTA,1486.00,15.71,63.0,H-MC,15.61,165172.0,38972.0,7201.0,2.41,30.88,4.36,36.59,108.0,5.41,1.20,47.32,XY24,NTT,HEALTHCARE
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,33.44,93696.0,-19853.0,59900.0,2.35,-17.48,63.93,35.27,268.0,-0.33,0.68,129.32,XR,NTT,HOTELS
62,REPCOHOME,880.00,-61.25,36.0,H-SC,8.47,142404.0,-62594.0,207839.0,2.29,-30.53,145.95,70.85,134.0,-0.30,1.03,16.99,XY24,NTT,FINANCE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INFY,2275.00,-19.24,45.0,H-LC,11.30,314324.0,1436.0,170238.0,-1.53,0.46,54.16,54.87,3.0,0.01,2.28,7.14,X40,BTT,IT
89,VOLTAS,1918.49,1.58,64.0,H-MC,4.99,213225.0,21483.0,74543.0,-1.33,11.20,34.96,50.08,99.0,0.29,1.55,18.76,XY25,ATH,AC
51,LTIM,7230.20,-6.07,52.0,H-LC,2.35,233235.0,-12916.0,92128.0,-1.15,-5.25,39.50,32.18,16.0,-0.14,1.69,28.90,X200,ATH,IT
37,HONAUT,58357.33,-20.78,42.0,H-SC,3.57,111705.0,-16233.0,63370.0,-1.13,-12.69,56.73,36.84,143.0,-0.26,0.81,14.36,X40N,ATH,ELECTRICAL
39,ICICIPRULI,790.00,-20.54,40.0,H-MC,2.17,136651.0,863.0,41884.0,-0.98,0.64,30.65,31.48,107.0,0.02,0.99,12.83,X40,ATH,INSURANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TTKPRESTIG,770.0,106.62,69.0,M-SC,6.98,89034.0,-11743.0,11833.0,-0.34,-11.65,13.29,0.09,245.0,-0.99,0.65,16.97,OX40N,NTT,DURABLES
83,UJJIVANSFB,60.0,84.36,45.0,H-SC,13.82,116505.0,-25974.0,45495.0,0.19,-18.23,39.05,13.70,163.0,-0.57,0.85,37.51,OX40N,NTT,BANKS
41,INDIGOPNTS,1408.0,116.52,53.0,M-SC,3.04,142005.0,-32554.0,32590.0,2.19,-18.65,22.95,0.02,221.0,-1.00,1.03,23.52,OX40N,NTT,PAINTS
48,KANSAINER,340.0,-68.16,55.0,H-SC,3.70,219150.0,-50517.0,86849.0,0.62,-18.73,39.63,13.47,138.0,-0.58,1.59,10.84,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,44.0,H-SC,3.08,113895.0,-29908.0,56651.0,-0.09,-20.80,49.74,18.60,149.0,-0.53,0.83,25.69,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,4998.00,-26.76,48.0,H-LC,14.72,291344.0,-54612.0,178477.0,-0.39,-15.79,61.26,35.80,1.0,-0.31,2.12,3.21,X40,BTT,IT
43,INFY,2275.00,-19.24,45.0,H-LC,11.30,314324.0,1436.0,170238.0,-1.53,0.46,54.16,54.87,3.0,0.01,2.28,7.14,X40,BTT,IT
45,ITC,452.00,-37.92,50.0,H-LC,3.29,199952.0,-186.0,21075.0,0.53,-0.09,10.54,10.44,4.0,-0.01,1.45,5.47,X40,NTT,FMCG
88,VBL,671.64,-9.80,55.0,H-LC,7.44,324742.0,8900.0,103755.0,1.96,2.82,31.95,35.67,5.0,0.09,2.36,17.06,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,49.0,H-LC,13.42,249024.0,-12595.0,131808.0,0.77,-4.81,52.93,45.57,7.0,-0.10,1.81,5.96,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,15.71,63.0,H-MC,15.61,165172.0,38972.0,7201.0,2.41,30.88,4.36,36.59,108.0,5.41,1.20,47.32,XY24,NTT,HEALTHCARE
34,HEROMOTOCO,5949.07,-0.14,75.0,H-MC,19.39,185360.0,34447.0,22855.0,-0.30,22.83,12.33,37.97,93.0,1.51,1.35,53.91,AR,ATH,AUTO
26,DIXON,18940.15,-29.21,69.0,H-MC,6.81,141624.0,26688.0,9900.0,0.85,23.22,6.99,31.83,81.0,2.70,1.03,46.80,X40N,ATH,IT
27,DMART,5201.00,-7.35,68.0,H-LC,14.33,231265.0,46784.0,23589.0,0.85,25.36,10.20,38.14,38.0,1.98,1.68,38.65,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,15.71,63.0,H-MC,15.61,165172.0,38972.0,7201.0,2.41,30.88,4.36,36.59,108.0,5.41,1.20,47.32,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,13.31,66.0,H-MC,3.23,195650.0,30980.0,36997.0,0.83,18.81,18.91,41.28,89.0,0.84,1.42,28.30,X40N,ATH,AC
49,KPIGREEN,731.35,5.57,46.0,H-SC,11.09,125786.0,489.0,58516.0,1.44,0.39,46.52,47.09,141.0,0.01,0.91,57.43,MH,ATH,POWER
89,VOLTAS,1918.49,1.58,64.0,H-MC,4.99,213225.0,21483.0,74543.0,-1.33,11.20,34.96,50.08,99.0,0.29,1.55,18.76,XY25,ATH,AC
32,HAVELLS,2069.16,0.20,64.0,H-MC,2.27,248461.0,730.0,74340.0,0.63,0.29,29.92,30.30,92.0,0.01,1.81,13.63,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-0.14,75.0,H-MC,19.39,185360.0,34447.0,22855.0,-0.30,22.83,12.33,37.97,93.0,1.51,1.35,53.91,AR,ATH,AUTO
54,NATIONALUM,247.44,-45.32,70.0,H-MC,0.53,108655.0,8151.0,22492.0,2.23,8.11,20.70,30.49,79.0,0.36,0.79,43.14,MH,ATH,METALS
90,WHIRLPOOL,2270.00,-40.54,57.0,M-SC,4.93,100425.0,8926.0,69826.0,0.40,9.76,69.53,86.07,223.0,0.13,0.73,45.84,XR,NTT,DURABLES
40,INDIAMART,4850.92,-51.84,44.0,H-SC,12.50,134525.0,11189.0,122579.0,-0.38,9.07,91.12,108.45,119.0,0.09,0.98,33.45,AR,ATH,MISC
91,WIPRO,420.00,-11.52,53.0,M-LC,7.15,155577.0,4632.0,104828.0,0.23,3.07,67.38,72.51,53.0,0.04,1.13,9.28,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAMMAANCAP,326.00,-172.73,70.0,M-SC,7.24,83820.0,-18390.0,111782.0,0.24,-17.99,133.36,91.37,208.0,-0.16,0.61,36.07,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-2.82,48.0,H-SC,16.22,90373.0,-10590.0,105122.0,1.94,-10.49,116.32,93.63,148.0,-0.10,0.66,34.61,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,33.44,93696.0,-19853.0,59900.0,2.35,-17.48,63.93,35.27,268.0,-0.33,0.68,129.32,XR,NTT,HOTELS
52,MASFIN,398.61,-16.44,57.0,H-SC,13.27,94725.0,-3255.0,24856.0,2.87,-3.32,26.24,22.05,152.0,-0.13,0.69,37.69,XR,ATH,FINANCE
54,NATIONALUM,247.44,-45.32,70.0,H-MC,0.53,108655.0,8151.0,22492.0,2.23,8.11,20.70,30.49,79.0,0.36,0.79,43.14,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,33.44,93696.0,-19853.0,59900.0,2.35,-17.48,63.93,35.27,268.0,-0.33,0.68,129.32,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-0.14,75.0,H-MC,19.39,185360.0,34447.0,22855.0,-0.30,22.83,12.33,37.97,93.0,1.51,1.35,53.91,AR,ATH,AUTO
60,RELAXO,1176.00,-30.97,64.0,H-SC,6.30,64070.0,-51468.0,86456.0,2.87,-44.55,134.94,30.28,136.0,-0.60,0.47,25.96,X40N,NTT,FOOTWEAR
27,DMART,5201.00,-7.35,68.0,H-LC,14.33,231265.0,46784.0,23589.0,0.85,25.36,10.20,38.14,38.0,1.98,1.68,38.65,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,13.31,66.0,H-MC,3.23,195650.0,30980.0,36997.0,0.83,18.81,18.91,41.28,89.0,0.84,1.42,28.30,X40N,ATH,AC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.26
1,25,42.47
2,50,72.09


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.47
SC    36.52
MC    26.03
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.78
X40      15.08
X40N     13.98
XY25     11.25
AR        9.16
XR        8.84
OX40N     7.23
X5K       2.24
MH        1.70
X200      1.69
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.49
H-MC    23.17
H-SC    21.87
M-SC    13.07
M-LC     6.96
M-MC     2.55
L-SC     1.58
L-LC     1.02
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.02,1.18,31.02
IT,13.01,-15.29,72.47
FINANCE,7.17,-22.27,67.35
BANKS,7.16,-15.83,65.01
PAINTS,5.86,-14.45,40.06
MISC,5.72,-16.71,83.61
ELECTRICAL,5.59,-7.10,66.13
AUTO,4.47,-9.99,57.84
AC,3.68,4.49,27.86


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2815208.0,23
AR,1219141.0,10
XR,1078601.0,13
X40,891379.0,10
X40N,508989.0,11
OX40N,464905.0,10
XY25,433344.0,8
SR,196876.0,2
X5K,130507.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2766822.0,27
M-SC,1447569.0,17
H-LC,1410446.0,18
H-MC,1190794.0,18
M-LC,420889.0,5
M-MC,327889.0,2
L-SC,244910.0,3
L-MC,59284.0,1
L-LC,43483.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,931109.0,6
M-SC,XY24,858079.0,7
H-SC,AR,664870.0,5
H-LC,X40,595277.0,5
H-SC,XR,522271.0,6
H-MC,XY24,467410.0,5
H-LC,AR,370678.0,2
M-MC,XY24,327889.0,2
H-MC,X40,217458.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
